In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn import pipeline, 
import xgboost as xgb
import warnings
import matplotlib.pyplot as plt
import plotly.express as px

warnings.filterwarnings('ignore')

Lecture des données

In [2]:
path_to_data = r"C:\Users\meghn\Documents\Dauphine\Master\M2 Finance Quantitative\Semestre 2\Machine Learning\data\\"
train_home_team_statistics_df = pd.read_csv(path_to_data+'Train_Data/train_home_team_statistics_df.csv', index_col=0)
train_away_team_statistics_df = pd.read_csv(path_to_data+'Train_Data/train_home_team_statistics_df.csv', index_col=0)
train_scores = pd.read_csv(path_to_data+'Y_train_1rknArQ.csv', index_col=0)

train_home = train_home_team_statistics_df.iloc[:,2:]
train_away = train_away_team_statistics_df.iloc[:,2:]

train_home.columns = 'HOME_' + train_home.columns
train_away.columns = 'AWAY_' + train_away.columns

train_data =  pd.concat([train_home,train_away],join='inner',axis=1)
train_scores = train_scores.loc[train_data.index]

train_data = train_data.replace({np.inf:np.nan,-np.inf:np.nan})


In [3]:
train_home

,HOME_TEAM_SHOTS_TOTAL_season_sum,HOME_TEAM_SHOTS_INSIDEBOX_season_sum,HOME_TEAM_SHOTS_OFF_TARGET_season_sum,HOME_TEAM_SHOTS_ON_TARGET_season_sum,HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum,HOME_TEAM_PASSES_season_sum,HOME_TEAM_SUCCESSFUL_PASSES_season_sum,HOME_TEAM_SAVES_season_sum,HOME_TEAM_CORNERS_season_sum,HOME_TEAM_FOULS_season_sum,...,HOME_TEAM_YELLOWCARDS_5_last_match_std,HOME_TEAM_REDCARDS_5_last_match_std,HOME_TEAM_OFFSIDES_5_last_match_std,HOME_TEAM_ATTACKS_5_last_match_std,HOME_TEAM_PENALTIES_5_last_match_std,HOME_TEAM_SUBSTITUTIONS_5_last_match_std,HOME_TEAM_BALL_SAFE_5_last_match_std,HOME_TEAM_DANGEROUS_ATTACKS_5_last_match_std,HOME_TEAM_INJURIES_5_last_match_std,HOME_TEAM_GOALS_5_last_match_std
ID,,,,,,,,,,,,,,,,,,,,,
0,3.0,2.0,5.0,2.0,1.0,2.0,2.0,5.0,3.0,6.0,...,3.0,0.0,6.0,0.0,10.0,8.0,7.0,2.0,4.0,3.0
1,6.0,8.0,3.0,6.0,5.0,8.0,7.0,10.0,6.0,8.0,...,4.0,0.0,4.0,3.0,10.0,0.0,1.0,2.0,8.0,4.0
2,4.0,2.0,5.0,2.0,8.0,1.0,1.0,2.0,2.0,7.0,...,4.0,5.0,6.0,3.0,6.0,7.0,2.0,3.0,2.0,4.0
3,7.0,5.0,5.0,6.0,6.0,9.0,9.0,2.0,2.0,0.0,...,4.0,0.0,1.0,8.0,8.0,5.0,5.0,5.0,NaN,6.0
4,3.0,3.0,2.0,3.0,4.0,4.0,3.0,4.0,4.0,7.0,...,1.0,0.0,2.0,5.0,8.0,7.0,2.0,6.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12298,4.0,2.0,3.0,4.0,7.0,4.0,4.0,3.0,2.0,2.0,...,4.0,0.0,2.0,5.0,0.0,5.0,0.0,0.0,NaN,4.0
12299,4.0,2.0,3.0,1.0,5.0,1.0,1.0,9.0,1.0,10.0,...,6.0,10.0,7.0,2.0,0.0,0.0,3.0,10.0,6.0,1.0
12300,4.0,3.0,5.0,3.0,5.0,1.0,1.0,6.0,1.0,8.0,...,7.0,8.0,0.0,7.0,0.0,0.0,3.0,1.0,3.0,2.0


On essaie de sélectionner les features qui nous sont pertinentes.

In [4]:
test1 = train_home_team_statistics_df.reset_index().merge(train_scores.reset_index(), on="ID").set_index('ID')

In [7]:
categorical_features = train_data.select_dtypes(include=['object']).columns.tolist()
numerical_features = train_data.select_dtypes(include=[np.number]).columns.tolist()

# Preprocessor for categorical and numerical data


# Splitting the data into training and testing sets with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Output sizes of the splits and a sample of processed data
X_train.shape, X_test.shape, y_train.shape, y_test.shape, X_train.head()

# categorical_features_clean = X_unique.select_dtypes(include=['object']).columns.tolist()
# numerical_features_clean = X_unique.select_dtypes(include=[np.number]).columns.tolist()

# # Redefine the preprocessor with the unique features
# preprocessor_clean = ColumnTransformer(
#     transformers=[
#         ('num', numerical_transformer, numerical_features_clean),
#         ('cat', categorical_transformer, categorical_features_clean)
#     ])

# # Update the pipeline with the new preprocessor
# pipeline_clean = Pipeline(steps=[('preprocessor', preprocessor_clean),
#                                  ('classifier', RandomForestClassifier(random_state=42))])

# # Splitting the data into training and testing sets with stratification
# X_train_clean, X_test_clean, y_train_clean, y_test_clean = train_test_split(
#     X_unique, y, test_size=0.2, random_state=42, stratify=y['HOME_WINS'])  # Assume 'HOME_WINS' for stratification example

# # Output sizes of the splits and display a sample of the processed training data using the pipeline
# X_train_clean.shape, X_test_clean.shape, y_train_clean.shape, y_test_clean.shape, X_train_clean.head()


,HOME_WINS,DRAW,AWAY_WINS
ID,,,
0,0,0,1
1,0,1,0
2,0,0,1
3,1,0,0
4,0,1,0
...,...,...,...
12298,0,0,1
12299,0,0,1
12300,0,0,1


In [ ]:
# List of possible target names
target_names = ['HOME_WINS', 'DRAW', 'AWAY_WINS']

# Check if any of these targets are in the correlation matrix
target_in_corr = any(name in corr_matrix_test.columns for name in target_names)

# Display whether targets are in the correlation matrix and handle the next steps accordingly
target_in_corr

In [6]:
test1.to_excel("train_test.xlsx")

In [10]:
std_sum = [col for col in test1 if 'std' in col or 'sum' in col]
grouped = test1.drop(columns=std_sum).groupby(by=["LEAGUE", "TEAM_NAME"]).sum()

In [15]:
grouped

TEAM_SHOTS_TOTAL_season_average  \
LEAGUE     TEAM_NAME                                                   
Bundesliga Bayer 04 Leverkusen                                 236.0   
           Borussia Dortmund                                   205.0   
           Borussia Mönchengladbach                            198.0   
           DSC Arminia Bielefeld                                 2.0   
           Eintracht Frankfurt                                 192.0   
...                                                              ...   
Superliga  Tigre                                                57.0   
           Unión Santa Fe                                      114.0   
           Vejle                                                67.0   
           Vendsyssel                                            8.0   
           Vélez Sarsfield                                      75.0   

                                     TEAM_SHOTS_INSIDEBOX_season_average  \
LEAGUE     TEAM_NAME                                                       
Bundesliga Bayer 04 Leverkusen                                     226.0   
           Borussia Dortmund                                       240.0   
           Borussia Mönchengladbach                                236.0   
           DSC Arminia Bielefeld                                     0.0   
           Eintracht Frankfurt                                     245.0   
...                                                                  ...   
Superliga  Tigre                                                    34.0   
           Unión Santa Fe                                           96.0   
           Vejle                                                    57.0   
           Vendsyssel                                                0.0   
           Vélez Sarsfield                                          52.0   

                                     TEAM_SHOTS_OFF_TARGET_season_average  \
LEAGUE     TEAM_NAME                                                        
Bundesliga Bayer 04 Leverkusen                                      183.0   
           Borussia Dortmund                                        106.0   
           Borussia Mönchengladbach                                  96.0   
           DSC Arminia Bielefeld                                      8.0   
           Eintracht Frankfurt                                      203.0   
...                                                                   ...   
Superliga  Tigre                                                     29.0   
           Unión Santa Fe                                           103.0   
           Vejle                                                     70.0   
           Vendsyssel                                                13.0   
           Vélez Sarsfield                                           90.0   

                                     TEAM_SHOTS_ON_TARGET_season_average  \
LEAGUE     TEAM_NAME                                                       
Bundesliga Bayer 04 Leverkusen                                     235.0   
           Borussia Dortmund                                       281.0   
           Borussia Mönchengladbach                                246.0   
           DSC Arminia Bielefeld                                     3.0   
           Eintracht Frankfurt                                     209.0   
...                                                                  ...   
Superliga  Tigre                                                    53.0   
           Unión Santa Fe                                           83.0   
           Vejle                                                   120.0   
           Vendsyssel                                               41.0   
           Vélez Sarsfield                                          84.0   

                                     TEAM_SHOTS_OUTSIDEBOX_season_average  \
LEAGUE     TEAM_NAME                       

In [16]:
grouped.to_excel("groupby.xlsx")

In [11]:
corr_home = grouped.iloc[:,2:].corr(numeric_only=False)
# cols_to_keep = train_scores.columns.to_list()

In [36]:
first_one = corr_home[["HOME_WINS", "AWAY_WINS", "DRAW"]] > 0.1

In [37]:
first_one

,HOME_WINS,AWAY_WINS,DRAW
TEAM_SHOTS_OFF_TARGET_season_average,True,True,True
TEAM_SHOTS_ON_TARGET_season_average,True,False,True
TEAM_SHOTS_OUTSIDEBOX_season_average,True,True,True
TEAM_PASSES_season_average,True,False,True
TEAM_SUCCESSFUL_PASSES_season_average,True,False,True
TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average,True,False,True
TEAM_SAVES_season_average,True,True,True
TEAM_CORNERS_season_average,True,False,True
TEAM_BALL_POSSESSION_season_average,True,False,True
TEAM_FOULS_season_average,True,True,True


: 

In [27]:
# sorted_mat = corr_home.unstack().sort_values() 
upper_corr_mat = corr_home.where( 
    np.triu(np.ones(corr_home.shape), k=1).astype(bool)) 
unique_corr_pairs = upper_corr_mat.unstack().dropna() 
sorted_mat = pd.DataFrame(abs(unique_corr_pairs.sort_values()))

In [28]:
sorted_mat

0
TEAM_GAME_LOST_season_average                      TEAM_GAME_WON_season_average                      0.232919
                                                   TEAM_GOALS_season_average                         0.202235
                                                   TEAM_SHOTS_ON_TARGET_season_average               0.150268
                                                   TEAM_BALL_POSSESSION_season_average               0.134760
                                                   TEAM_SUCCESSFUL_PASSES_season_average             0.129450
...                                                                                                       ...
TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_... TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average  0.980975
TEAM_PASSES_5_last_match_average                   TEAM_SUCCESSFUL_PASSES_season_average             0.983156
TEAM_SUCCESSFUL_PASSES_5_last_match_average        TEAM_SUCCESSFUL_PASSES_season_average             0.984776
TEAM_SUCCESSFUL_PASSES_season_average              TEAM_PASSES_season_average                        0.992307
TEAM_SUCCESSFUL_PASSES_5_last_match_average        TEAM_PASSES_5_last_match_average                  0.997094

[1275 rows x 1 columns]

In [30]:
sorted_mat['decile rank'] = pd.qcut(sorted_mat[0], 10, 
                               labels = False) 

In [31]:
sorted_mat

0  \
TEAM_GAME_LOST_season_average                      TEAM_GAME_WON_season_average                      0.232919   
                                                   TEAM_GOALS_season_average                         0.202235   
                                                   TEAM_SHOTS_ON_TARGET_season_average               0.150268   
                                                   TEAM_BALL_POSSESSION_season_average               0.134760   
                                                   TEAM_SUCCESSFUL_PASSES_season_average             0.129450   
...                                                                                                       ...   
TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_... TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average  0.980975   
TEAM_PASSES_5_last_match_average                   TEAM_SUCCESSFUL_PASSES_season_average             0.983156   
TEAM_SUCCESSFUL_PASSES_5_last_match_average        TEAM_SUCCESSFUL_PASSES_season_average             0.984776   
TEAM_SUCCESSFUL_PASSES_season_average              TEAM_PASSES_season_average                        0.992307   
TEAM_SUCCESSFUL_PASSES_5_last_match_average        TEAM_PASSES_5_last_match_average                  0.997094   

                                                                                                     decile rank  
TEAM_GAME_LOST_season_average                      TEAM_GAME_WON_season_average                                1  
                                                   TEAM_GOALS_season_average                                   0  
                                                   TEAM_SHOTS_ON_TARGET_season_average                         0  
                                                   TEAM_BALL_POSSESSION_season_average                         0  
                                                   TEAM_SUCCESSFUL_PASSES_season_average                       0  
...                                                                                                          ...  
TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_... TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average            9  
TEAM_PASSES_5_last_match_average                   TEAM_SUCCESSFUL_PASSES_season_average                       9  
TEAM_SUCCESSFUL_PASSES_5_last_match_average        TEAM_SUCCESSFUL_PASSES_season_average                       9  
TEAM_SUCCESSFUL_PASSES_season_average              TEAM_PASSES_season_average                                  9  
TEAM_SUCCESSFUL_PASSES_5_last_match_average        TEAM_PASSES_5_last_match_average                            9  

[1275 rows x 2 columns]

In [32]:
sorted_mat.to_excel("matrice_corr_home.xlsx")

In [18]:
fig = px.imshow(corr_home, contrast_rescaling='infer')
fig.show()

on va essayer de trouver quels champs sont +/- facteurs de succès à domicile

In [4]:
train_away_team_statistics_df

,LEAGUE,TEAM_NAME,TEAM_SHOTS_TOTAL_season_sum,TEAM_SHOTS_INSIDEBOX_season_sum,TEAM_SHOTS_OFF_TARGET_season_sum,TEAM_SHOTS_ON_TARGET_season_sum,TEAM_SHOTS_OUTSIDEBOX_season_sum,TEAM_PASSES_season_sum,TEAM_SUCCESSFUL_PASSES_season_sum,TEAM_SAVES_season_sum,...,TEAM_YELLOWCARDS_5_last_match_std,TEAM_REDCARDS_5_last_match_std,TEAM_OFFSIDES_5_last_match_std,TEAM_ATTACKS_5_last_match_std,TEAM_PENALTIES_5_last_match_std,TEAM_SUBSTITUTIONS_5_last_match_std,TEAM_BALL_SAFE_5_last_match_std,TEAM_DANGEROUS_ATTACKS_5_last_match_std,TEAM_INJURIES_5_last_match_std,TEAM_GOALS_5_last_match_std
ID,,,,,,,,,,,,,,,,,,,,,
0,Ligue 1,Toulouse,3.0,2.0,5.0,2.0,1.0,2.0,2.0,5.0,...,3.0,0.0,6.0,0.0,10.0,8.0,7.0,2.0,4.0,3.0
1,Ligue 2,Brest,6.0,8.0,3.0,6.0,5.0,8.0,7.0,10.0,...,4.0,0.0,4.0,3.0,10.0,0.0,1.0,2.0,8.0,4.0
2,Serie A,Sampdoria,4.0,2.0,5.0,2.0,8.0,1.0,1.0,2.0,...,4.0,5.0,6.0,3.0,6.0,7.0,2.0,3.0,2.0,4.0
3,League One,Coventry City,7.0,5.0,5.0,6.0,6.0,9.0,9.0,2.0,...,4.0,0.0,1.0,8.0,8.0,5.0,5.0,5.0,NaN,6.0
4,Premier League,Wolverhampton Wanderers,3.0,3.0,2.0,3.0,4.0,4.0,3.0,4.0,...,1.0,0.0,2.0,5.0,8.0,7.0,2.0,6.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12298,League One,Southend United,4.0,2.0,3.0,4.0,7.0,4.0,4.0,3.0,...,4.0,0.0,2.0,5.0,0.0,5.0,0.0,0.0,NaN,4.0
12299,Liga Portugal,Boavista,4.0,2.0,3.0,1.0,5.0,1.0,1.0,9.0,...,6.0,10.0,7.0,2.0,0.0,0.0,3.0,10.0,6.0,1.0
12300,Bundesliga,Schalke 04,4.0,3.0,5.0,3.0,5.0,1.0,1.0,6.0,...,7.0,8.0,0.0,7.0,0.0,0.0,3.0,1.0,3.0,2.0
